In [1]:
from torch import nn , save , load
import torchvision
import torch
import numpy as np
from tqdm.notebook import tqdm
import torch.nn.functional as F

In [21]:
import os
import cv2
import numpy as np

os.remove("dataset/README")

In [80]:
# pip install natsort 
from natsort import natsorted

all_images = []
all_labels = []

entries = os.listdir("dataset")
# Natsort is used to sort the images in the order of their names which the normal sort fails to do
entries = natsorted(entries)
for i,entry in enumerate(entries):
    images = os.listdir("dataset/"+entry)
    images = natsorted(images)
    for image in images:
        # Reading the image and converting it to grayscale
        img = cv2.imread("dataset/"+entry+"/"+image,cv2.IMREAD_GRAYSCALE)
        # Flastten / normalize the image
        all_images.append(img.flatten()/255)
        all_labels.append(i+1)

# Generating Data matrix
all_images = np.array(all_images)
all_labels = np.array(all_labels)
all_labels=all_labels.reshape(all_labels.shape[0],-1)

print(f"Images shape: {all_images.shape}")
print(f"Labels shape: {all_labels.shape}")

Images shape: (400, 10304)
Labels shape: (400, 1)


In [81]:
# Splitting the data into train and test (Odd images for training and even images for testing) <<-- This is a bad way to do it
Xtrain = all_images[0::2]
Ytrain = all_labels[0::2]
Xtest = all_images[1::2]
Ytest = all_labels[1::2]

print(f"Xtrain shape: {Xtrain.shape}")
print(f"Ytrain shape: {Ytrain.shape}")
print(f"Xtest shape: {Xtest.shape}")
print(f"Ytest shape: {Ytest.shape}")

Xtrain shape: (200, 10304)
Ytrain shape: (200, 1)
Xtest shape: (200, 10304)
Ytest shape: (200, 1)


In [160]:
Xtrain_transposed = Xtrain.transpose()
means = [np.mean(row) for row in Xtrain_transposed]
Z = Xtrain - means
cov = 1/len(Xtrain) * np.matmul(Z,Z.transpose())
eig_values, eig_vectors = np.linalg.eigh(cov)

In [161]:
eig_values = np.flip(eig_values)

In [162]:
def get_dimesions(eigen_values, alpha):
    total = eig_values.sum()
    sum=0
    for i,val in enumerate(eigen_values):
        sum+=val
        if sum/total >= alpha:
            return i
    return i

In [167]:
alphas = [0.8,0.85,0.9,0.95]
r = [get_dimesions(eig_values,alpha)+1 for alpha in alphas]
r

[36, 51, 76, 115]

In [168]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import classification_report
# idx = eig_values.argsort()[::-1]  
# eig_values = eig_values[idx]
# eig_vectors = eig_vectors[:,idx]
# for x in range(len(r)):
    
#     p_train_data = np.matmul(eig_vectors[:, 0:r[x]].T, Xtrain.T).T
#     p_test_data = np.matmul(eig_vectors[:, 0:r[x]].T, Xtest.T).T

#     neigh = KNeighborsClassifier(n_neighbors=1)
#     neigh.fit(p_train_data, Ytrain)

#     y_pred=neigh.predict(p_test_data)

#     print('accuracy for alpha = ',alphas[x])
#     print(classification_report(Ytest, y_pred))